In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
page = requests.get('https://www.imdb.com/chart/toptv/?sort=rk,asc&mode=simple&page=1')
soup = BeautifulSoup(page.content, 'html.parser')

In [3]:
# CODE FOR WEBSCRAPING LIST OF TOP RATED SHOWS
movie_titles = soup.find_all(class_='titleColumn')

In [4]:
# learned about dict comprehension, created dict comprehension for top 250 rated show, for codes and titles
code_title_dict = {mov.a['href'].split('/')[2]:mov.a.text for mov in movie_titles}

In [5]:
# check that len of dict is correspondent with length of imdb source
len(code_title_dict)

250

In [6]:
# created a list called codes which contians all the keys from code_title_dict
codes =[]
for k in code_title_dict.keys():
    codes.append(k)

In [7]:
# CODE FOR CALLING TV API
def tv_api_call(imdbcode):
#     print(imdbcode) this is an error check, checking which code fails

    url =  'http://api.tvmaze.com/lookup/shows?imdb='+imdbcode
    
    response = requests.get(url)
    
    if response.status_code != 200:
        data = None
    else:
        data = json.loads(response.text)
#     created if statement for status code checks, if it doesnt work, then data is none
#     if it does work then data loads the response as a json
    return data

In [8]:
def parse_shows(shows):
    show_dict = []
    for ind, result in enumerate(shows):
        each_show = {'imdb_id':result['externals']['imdb'],
                     'title':result['name'],
                     'type':result['type'],
                     'genre':result['genres'],
                     'runtime':result['runtime'], 
                     'rating':result['rating']['average']}
        
        if  result['webChannel'] != None:
            each_show['webChannel'] = result['webChannel']['name']
        else:
            each_show['webChannel'] = False
        
        
        if result['network'] != None:
            each_show['network'] = result['network']['name']
        else:
            each_show['network'] = False  
            
        show_dict.append(each_show)
    return show_dict

# created function to parse through the api calls and return a list of dictionaries with the defined keys
# created the if statements for situations when 'webChannel' & 'network' keys in shows dict have None values

In [ ]:
# created list 'responses' for list of dicts of each show
# created list 'no_work' for list of show titles which got a status code of not 200 and were not able to be pulled
responses = []
no_work = []
for value in codes:
    called_shows = tv_api_call(value)
    if called_shows != None:
        responses.append(called_shows)
    else:
        no_work.append(value)

In [ ]:
second_call = parse_shows(responses)
# called parse_shows function with responses as argument, for the list of dicts

In [ ]:
net_show = pd.DataFrame(second_call)
# turned list of dicts into df, with keys of the dicts being column names and values of
# each dict becoming single row